# Notebook 5: Identify Route Parish Intersections

## Introduction to Noteboook 4

Why are route-parish intersections important? https://cw.fel.cvut.cz/b181/_media/courses/cg/lectures/09-intersect.pdf



In [77]:
import osmnx as ox, csv

with open('overpass-api.csv', mode='r') as infile:
    reader = csv.reader(infile)
    overpass_api = {rows[0]:rows[1] for rows in reader}

ox.config(
    log_console=False, 
    use_cache=True, 
    log_file=True,
    overpass_endpoint=overpass_api['main']
)

In [78]:
%%time
if 'G_proj' not in globals():
    G_proj = ox.load_graphml('data/g_projected.graphml')

if 'G' not in globals():
    G = ox.load_graphml('data/g_unprojected.graphml')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


## Data sources

In [79]:
import pandas as pd, geopandas as gpd, folium

pd.options.display.float_format = '{:.10f}'.format

### Paired cache GDF, parishes GDF

In [80]:
parishes_gdf = gpd.read_file('data/parishes_gdf.gpkg')
paired_cache_df = pd.read_pickle('data/paired_cache_df.pickle')

In [81]:
parishes_gdf

,GID_0,NAME_0,GID_1,NAME_1,GID_2,NAME_2,GID_3,NAME_3,GID_4,NAME_4,VARNAME_4,TYPE_4,ENGTYPE_4,CC_4,area_sqm,centroid_x,centroid_y,geometry
0,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.1_1,Central Division,UGA.16.1.1.1_1,Bukesa,None,Parish,Parish,None,559768.3975113713,451524.4166484834,35906.9724917396,"MULTIPOLYGON (((32.56321 0.31958, 32.56292 0.3..."
1,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.1_1,Central Division,UGA.16.1.1.2_1,Civic Centre,None,Parish,Parish,None,1209455.8146584462,453884.4876301024,34880.4453069278,"MULTIPOLYGON (((32.57724 0.30992, 32.57730 0.3..."
2,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.1_1,Central Division,UGA.16.1.1.3_1,Industrial Area,None,Parish,Parish,None,1776410.1888784864,455890.3457818956,35463.5391795589,"MULTIPOLYGON (((32.60664 0.31585, 32.60598 0.3..."
3,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.1_1,Central Division,UGA.16.1.1.4_1,Kagugube,None,Parish,Parish,None,502708.6719317246,452030.8561209833,36273.6179740557,"MULTIPOLYGON (((32.57084 0.32427, 32.57058 0.3..."
4,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.1_1,Central Division,UGA.16.1.1.5_1,Kamwokya I,None,Parish,Parish,None,411428.6768850113,454564.2625147337,38168.9266840896,"MULTIPOLYGON (((32.58500 0.34132, 32.58651 0.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.5_1,Rubaga Division,UGA.16.1.5.9_1,Nakulabye,None,Parish,Parish,None,1026084.8535207562,450559.2039879204,36410.2678852486,"MULTIPOLYGON (((32.54860 0.33223, 32.55151 0.3..."
92,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.5_1,Rubaga Division,UGA.16.1.5.10_1,Namirembe,None,Parish,Parish,None,1859599.6021243404,450911.3264704964,35032.9390910053,"MULTIPOLYGON (((32.56114 0.30923, 32.56108 0.3..."
93,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.5_1,Rubaga Division,UGA.16.1.5.11_1,Nateete,None,Parish,Parish,None,1804795.3528157366,448176.4503960741,33820.1469633023,"MULTIPOLYGON (((32.54007 0.30528, 32.53991 0.3..."
94,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.5_1,Rubaga Division,UGA.16.1.5.12_1,Ndeeba,None,Parish,Parish,None,1962463.6518844874,451649.7161646702,33182.2916360861,"MULTIPOLYGON (((32.57188 0.29548, 32.57166 0.2..."


In [82]:
paired_cache_df

,r_node,parish_name,r_node_lat,r_node_lon,t_node,t_node_lat,t_node_lon,d3_euc,d3_shapely,path_node_list,path_distances,d3_path_sum,d3_edge_attrs,d_total
0,6224792359,Kabowa,0.2873915000,32.5623252000,5572008964,0.2883498000,32.5594612000,246.4599184290,764.8577704742,"[6224792359, 557330540, 557330539, 557330554, ...","[14.009421458485344, 27.723009203707186, 74.93...",764.8577704742,766.7720000000,863.0244679421
1,6241486007,Kabowa,0.2910965000,32.5589864000,5572008964,0.2883498000,32.5594612000,306.8949496724,489.8071930412,"[6241486007, 6241486005, 6100868024, 493323985...","[6.467357789954945, 9.049813483031556, 16.9813...",489.8071930412,491.3860000000,540.4074311963
2,2085397292,Kabowa,0.2852278000,32.5606116000,5572008964,0.2883498000,32.5594612000,367.1917534631,571.3800720441,"[2085397292, 2085397298, 2085397289, 208539729...","[24.769779727776356, 24.46113682821943, 15.947...",571.3800720441,573.4790000000,641.7737756892
3,7100592251,Kabowa,0.2925402000,32.5509825000,5572008964,0.2883498000,32.5594612000,1079.1479627214,1401.1835808965,"[7100592251, 7100592252, 7100592253, 710059225...","[4.735116716464567, 15.49655807067614, 9.96821...",1401.1835808965,1402.8320000000,1443.0140758016
4,6229407820,Kabowa,0.2911225000,32.5589936000,5572008964,0.2883498000,32.5594612000,304.2698779717,492.7906517788,"[6229407820, 6241486007, 6241486005, 610086802...","[2.9834587375602957, 6.467357789954945, 9.0498...",492.7906517788,494.3860000000,552.0968891205
5,475033488,Kisugu,0.3109017000,32.6098430000,7401065633,0.3076673000,32.6249390000,1694.6835003641,2617.9292724860,"[475033488, 1583799847, 475033491, 3481960627,...","[34.857797441936704, 18.888149019910713, 9.026...",2617.9292724860,2622.6930000000,2729.7292195052
6,579993375,Kisugu,0.3082407000,32.6077652000,7401065633,0.3076673000,32.6249390000,1864.5203716102,3031.4086806217,"[579993375, 579993374, 579993373, 6226994993, ...","[19.542746737921256, 16.795537250722806, 8.728...",3031.4086806217,3037.0040000000,3151.0671117949
7,579993404,Kisugu,0.3097587000,32.6091322000,7401065633,0.3076673000,32.6249390000,1745.8336543244,2730.6216586954,"[579993404, 1583799967, 1583799788, 1583799935...","[32.71856677381537, 26.624433957601738, 25.730...",2730.6216586954,2735.6840000000,2840.9248157841
8,1583799787,Kisugu,0.3100471000,32.6100813000,7401065633,0.3076673000,32.6249390000,1640.4063742119,2677.1014087277,"[1583799787, 6227589978, 1583800118, 158380034...","[9.721294101490457, 7.796506912348756, 12.5149...",2677.1014087277,2682.0780000000,2782.0532660583
9,8144996943,Kisugu,0.3033524000,32.6059574000,8003448502,0.3007283000,32.5890897000,1892.8943434713,2774.3196406058,"[8144996943, 8144996944, 579992620, 579992609,...","[46.50197410980276, 51.71860469243541, 56.8267...",2774.3196406058,2776.6460000000,2802.9625482335


## Create GDF from paired cache DF

In [83]:
import yaml

with open("proj_crs.yml", "r") as stream:
    try:
        epsg_dict = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
proj_epsg_str = str(epsg_dict).replace("{","").replace("}", "").replace("'","").replace(" ","")

print('Projected graph EPSG code:',proj_epsg_str)

Projected graph EPSG code: EPSG:32636


In [84]:
from shapely.geometry import LineString, Point

paired_cache_df['geometry'] = ''
for index, row in paired_cache_df.iterrows():
    path_node_list = str(row['path_node_list']).strip('[]').split(',')
    path_node_list = [ int(item.strip()) for item in path_node_list ]
    coords_list = [(G_proj.nodes[node]['x'], G_proj.nodes[node]['y']) for node in path_node_list ]
    path_line = LineString(coords_list)
    paired_cache_df.loc[index, 'geometry'] = path_line        

In [85]:
paired_cache_gdf = gpd.GeoDataFrame(paired_cache_df, geometry='geometry', crs=proj_epsg_str)

paired_cache_gdf

,r_node,parish_name,r_node_lat,r_node_lon,t_node,t_node_lat,t_node_lon,d3_euc,d3_shapely,path_node_list,path_distances,d3_path_sum,d3_edge_attrs,d_total,geometry
0,6224792359,Kabowa,0.2873915000,32.5623252000,5572008964,0.2883498000,32.5594612000,246.4599184290,764.8577704742,"[6224792359, 557330540, 557330539, 557330554, ...","[14.009421458485344, 27.723009203707186, 74.93...",764.8577704742,766.7720000000,863.0244679421,"LINESTRING (451297.885 31766.331, 451300.567 3..."
1,6241486007,Kabowa,0.2910965000,32.5589864000,5572008964,0.2883498000,32.5594612000,306.8949496724,489.8071930412,"[6241486007, 6241486005, 6100868024, 493323985...","[6.467357789954945, 9.049813483031556, 16.9813...",489.8071930412,491.3860000000,540.4074311963,"LINESTRING (450926.369 32175.872, 450924.132 3..."
2,2085397292,Kabowa,0.2852278000,32.5606116000,5572008964,0.2883498000,32.5594612000,367.1917534631,571.3800720441,"[2085397292, 2085397298, 2085397289, 208539729...","[24.769779727776356, 24.46113682821943, 15.947...",571.3800720441,573.4790000000,641.7737756892,"LINESTRING (451107.191 31527.178, 451085.692 3..."
3,7100592251,Kabowa,0.2925402000,32.5509825000,5572008964,0.2883498000,32.5594612000,1079.1479627214,1401.1835808965,"[7100592251, 7100592252, 7100592253, 710059225...","[4.735116716464567, 15.49655807067614, 9.96821...",1401.1835808965,1402.8320000000,1443.0140758016,"LINESTRING (450035.726 32335.484, 450039.866 3..."
4,6229407820,Kabowa,0.2911225000,32.5589936000,5572008964,0.2883498000,32.5594612000,304.2698779717,492.7906517788,"[6229407820, 6241486007, 6241486005, 610086802...","[2.9834587375602957, 6.467357789954945, 9.0498...",492.7906517788,494.3860000000,552.0968891205,"LINESTRING (450927.171 32178.746, 450926.369 3..."
5,475033488,Kisugu,0.3109017000,32.6098430000,7401065633,0.3076673000,32.6249390000,1694.6835003641,2617.9292724860,"[475033488, 1583799847, 475033491, 3481960627,...","[34.857797441936704, 18.888149019910713, 9.026...",2617.9292724860,2622.6930000000,2729.7292195052,"LINESTRING (456585.591 34364.785, 456611.507 3..."
6,579993375,Kisugu,0.3082407000,32.6077652000,7401065633,0.3076673000,32.6249390000,1864.5203716102,3031.4086806217,"[579993375, 579993374, 579993373, 6226994993, ...","[19.542746737921256, 16.795537250722806, 8.728...",3031.4086806217,3037.0040000000,3151.0671117949,"LINESTRING (456354.371 34070.666, 456373.700 3..."
7,579993404,Kisugu,0.3097587000,32.6091322000,7401065633,0.3076673000,32.6249390000,1745.8336543244,2730.6216586954,"[579993404, 1583799967, 1583799788, 1583799935...","[32.71856677381537, 26.624433957601738, 25.730...",2730.6216586954,2735.6840000000,2840.9248157841,"LINESTRING (456506.492 34238.449, 456532.709 3..."
8,1583799787,Kisugu,0.3100471000,32.6100813000,7401065633,0.3076673000,32.6249390000,1640.4063742119,2677.1014087277,"[1583799787, 6227589978, 1583800118, 158380034...","[9.721294101490457, 7.796506912348756, 12.5149...",2677.1014087277,2682.0780000000,2782.0532660583,"LINESTRING (456612.105 34270.323, 456606.586 3..."
9,8144996943,Kisugu,0.3033524000,32.6059574000,8003448502,0.3007283000,32.5890897000,1892.8943434713,2774.3196406058,"[8144996943, 8144996944, 579992620, 579992609,...","[46.50197410980276, 51.71860469243541, 56.8267...",2774.3196406058,2776.6460000000,2802.9625482335,"LINESTRING (456153.187 33530.356, 456137.242 3..."


### Change GDF projection

In [86]:
paired_cache_proj_gdf = paired_cache_gdf.to_crs(proj_epsg_str)
parishes_proj_gdf = parishes_gdf.to_crs(proj_epsg_str)

### Add two new columns

In [87]:
paired_cache_proj_gdf['parish_route'] = ''
paired_cache_proj_gdf['parish_num'] = 0

In [91]:
paired_cache_proj_gdf

,r_node,parish_name,r_node_lat,r_node_lon,t_node,t_node_lat,t_node_lon,d3_euc,d3_shapely,path_node_list,path_distances,d3_path_sum,d3_edge_attrs,d_total,geometry,parish_route,parish_num
0,6224792359,Kabowa,0.2873915000,32.5623252000,5572008964,0.2883498000,32.5594612000,246.4599184290,764.8577704742,"[6224792359, 557330540, 557330539, 557330554, ...","[14.009421458485344, 27.723009203707186, 74.93...",764.8577704742,766.7720000000,863.0244679421,"LINESTRING (451297.885 31766.331, 451300.567 3...",,0
1,6241486007,Kabowa,0.2910965000,32.5589864000,5572008964,0.2883498000,32.5594612000,306.8949496724,489.8071930412,"[6241486007, 6241486005, 6100868024, 493323985...","[6.467357789954945, 9.049813483031556, 16.9813...",489.8071930412,491.3860000000,540.4074311963,"LINESTRING (450926.369 32175.872, 450924.132 3...",,0
2,2085397292,Kabowa,0.2852278000,32.5606116000,5572008964,0.2883498000,32.5594612000,367.1917534631,571.3800720441,"[2085397292, 2085397298, 2085397289, 208539729...","[24.769779727776356, 24.46113682821943, 15.947...",571.3800720441,573.4790000000,641.7737756892,"LINESTRING (451107.191 31527.178, 451085.692 3...",,0
3,7100592251,Kabowa,0.2925402000,32.5509825000,5572008964,0.2883498000,32.5594612000,1079.1479627214,1401.1835808965,"[7100592251, 7100592252, 7100592253, 710059225...","[4.735116716464567, 15.49655807067614, 9.96821...",1401.1835808965,1402.8320000000,1443.0140758016,"LINESTRING (450035.726 32335.484, 450039.866 3...",,0
4,6229407820,Kabowa,0.2911225000,32.5589936000,5572008964,0.2883498000,32.5594612000,304.2698779717,492.7906517788,"[6229407820, 6241486007, 6241486005, 610086802...","[2.9834587375602957, 6.467357789954945, 9.0498...",492.7906517788,494.3860000000,552.0968891205,"LINESTRING (450927.171 32178.746, 450926.369 3...",,0
5,475033488,Kisugu,0.3109017000,32.6098430000,7401065633,0.3076673000,32.6249390000,1694.6835003641,2617.9292724860,"[475033488, 1583799847, 475033491, 3481960627,...","[34.857797441936704, 18.888149019910713, 9.026...",2617.9292724860,2622.6930000000,2729.7292195052,"LINESTRING (456585.591 34364.785, 456611.507 3...",,0
6,579993375,Kisugu,0.3082407000,32.6077652000,7401065633,0.3076673000,32.6249390000,1864.5203716102,3031.4086806217,"[579993375, 579993374, 579993373, 6226994993, ...","[19.542746737921256, 16.795537250722806, 8.728...",3031.4086806217,3037.0040000000,3151.0671117949,"LINESTRING (456354.371 34070.666, 456373.700 3...",,0
7,579993404,Kisugu,0.3097587000,32.6091322000,7401065633,0.3076673000,32.6249390000,1745.8336543244,2730.6216586954,"[579993404, 1583799967, 1583799788, 1583799935...","[32.71856677381537, 26.624433957601738, 25.730...",2730.6216586954,2735.6840000000,2840.9248157841,"LINESTRING (456506.492 34238.449, 456532.709 3...",,0
8,1583799787,Kisugu,0.3100471000,32.6100813000,7401065633,0.3076673000,32.6249390000,1640.4063742119,2677.1014087277,"[1583799787, 6227589978, 1583800118, 158380034...","[9.721294101490457, 7.796506912348756, 12.5149...",2677.1014087277,2682.0780000000,2782.0532660583,"LINESTRING (456612.105 34270.323, 456606.586 3...",,0
9,8144996943,Kisugu,0.3033524000,32.6059574000,8003448502,0.3007283000,32.5890897000,1892.8943434713,2774.3196406058,"[8144996943, 8144996944, 579992620, 579992609,...","[46.50197410980276, 51.71860469243541, 56.8267...",2774.3196406058,2776.6460000000,2802.9625482335,"LINESTRING (456153.187 33530.356, 456137.242 3...",,0


References:
1. Convex Hull: https://geopandas.org/docs/user_guide/geometric_manipulations.html#GeoSeries.convex_hull
2. Distance between geoseries: https://geopandas.org/docs/reference/api/geopandas.GeoSeries.distance.html#geopandas.GeoSeries.distance
3. Geoseries contains: https://geopandas.org/docs/reference/api/geopandas.GeoSeries.contains.html#

## Set Parish GDF index to `NAME_4` (parish name)

In [88]:
parishes_proj_gdf = parishes_proj_gdf.set_index(['NAME_4'])

In [89]:
parishes_proj_gdf.loc[['Kisugu']]

,GID_0,NAME_0,GID_1,NAME_1,GID_2,NAME_2,GID_3,NAME_3,GID_4,VARNAME_4,TYPE_4,ENGTYPE_4,CC_4,area_sqm,centroid_x,centroid_y,geometry
NAME_4,,,,,,,,,,,,,,,,,
Kisugu,UGA,Uganda,UGA.16_1,Kampala,UGA.16.1_1,Kampala,UGA.16.1.3_1,Makindye,UGA.16.1.3.11_1,None,Parish,Parish,None,1227491.9265254750,456230.4708387037,34104.6820534491,"MULTIPOLYGON (((456229.976 33480.017, 456139.1..."


## Identify route-parish intersections and parish routes

In [93]:
from shapely.geometry import Point
from tqdm import tqdm, notebook

# tqdm parameters
total_rows=paired_cache_proj_gdf.shape[0]

# route loop
for r_index, r_row in tqdm(paired_cache_proj_gdf.iterrows(), total=total_rows):

    # parish loop to build parish list
    # identifies parishes intersected by route
    parish_list = []
    for p_index, p_row in parishes_proj_gdf.iterrows():
        
        distance = r_row['geometry'].distance(p_row['geometry'].convex_hull)

        if distance == 0:
            parish_list.append(p_index)

    # go through path nodes to get right order of parishes (parish_route)
    path_node_list = str(r_row['path_node_list']).strip('[]').split(',')
    path_node_list = [ int(item.strip()) for item in path_node_list ]
    parish_route = []
    for node in path_node_list:
        node_point = Point(G_proj.nodes[node]['x'], G_proj.nodes[node]['y'])
        for parish in parish_list:
            if parishes_proj_gdf.loc[[parish]]['geometry'][0].convex_hull.contains(node_point):
                if parish not in parish_route:
                    parish_route.append(parish)
    #print('parish list:', parish_list)
    #print('parish route:', parish_route)
    #print('number of parishes:', len(parish_route))
    paired_cache_proj_gdf.at[r_index, 'parish_route'] = parish_route
    paired_cache_proj_gdf.at[r_index, 'parish_num'] = len(parish_route)
    

100%|██████████| 20/20 [00:06<00:00,  2.87it/s]


In [94]:
paired_cache_proj_gdf

,r_node,parish_name,r_node_lat,r_node_lon,t_node,t_node_lat,t_node_lon,d3_euc,d3_shapely,path_node_list,path_distances,d3_path_sum,d3_edge_attrs,d_total,geometry,parish_route,parish_num
0,6224792359,Kabowa,0.2873915000,32.5623252000,5572008964,0.2883498000,32.5594612000,246.4599184290,764.8577704742,"[6224792359, 557330540, 557330539, 557330554, ...","[14.009421458485344, 27.723009203707186, 74.93...",764.8577704742,766.7720000000,863.0244679421,"LINESTRING (451297.885 31766.331, 451300.567 3...","[Kabowa, Najjanankumbi Ii]",2
1,6241486007,Kabowa,0.2910965000,32.5589864000,5572008964,0.2883498000,32.5594612000,306.8949496724,489.8071930412,"[6241486007, 6241486005, 6100868024, 493323985...","[6.467357789954945, 9.049813483031556, 16.9813...",489.8071930412,491.3860000000,540.4074311963,"LINESTRING (450926.369 32175.872, 450924.132 3...",[Kabowa],1
2,2085397292,Kabowa,0.2852278000,32.5606116000,5572008964,0.2883498000,32.5594612000,367.1917534631,571.3800720441,"[2085397292, 2085397298, 2085397289, 208539729...","[24.769779727776356, 24.46113682821943, 15.947...",571.3800720441,573.4790000000,641.7737756892,"LINESTRING (451107.191 31527.178, 451085.692 3...","[Kabowa, Najjanankumbi Ii]",2
3,7100592251,Kabowa,0.2925402000,32.5509825000,5572008964,0.2883498000,32.5594612000,1079.1479627214,1401.1835808965,"[7100592251, 7100592252, 7100592253, 710059225...","[4.735116716464567, 15.49655807067614, 9.96821...",1401.1835808965,1402.8320000000,1443.0140758016,"LINESTRING (450035.726 32335.484, 450039.866 3...",[Kabowa],1
4,6229407820,Kabowa,0.2911225000,32.5589936000,5572008964,0.2883498000,32.5594612000,304.2698779717,492.7906517788,"[6229407820, 6241486007, 6241486005, 610086802...","[2.9834587375602957, 6.467357789954945, 9.0498...",492.7906517788,494.3860000000,552.0968891205,"LINESTRING (450927.171 32178.746, 450926.369 3...",[Kabowa],1
5,475033488,Kisugu,0.3109017000,32.6098430000,7401065633,0.3076673000,32.6249390000,1694.6835003641,2617.9292724860,"[475033488, 1583799847, 475033491, 3481960627,...","[34.857797441936704, 18.888149019910713, 9.026...",2617.9292724860,2622.6930000000,2729.7292195052,"LINESTRING (456585.591 34364.785, 456611.507 3...","[Kisugu, Industrial Area, Bogolobi]",3
6,579993375,Kisugu,0.3082407000,32.6077652000,7401065633,0.3076673000,32.6249390000,1864.5203716102,3031.4086806217,"[579993375, 579993374, 579993373, 6226994993, ...","[19.542746737921256, 16.795537250722806, 8.728...",3031.4086806217,3037.0040000000,3151.0671117949,"LINESTRING (456354.371 34070.666, 456373.700 3...","[Kisugu, Industrial Area, Bogolobi]",3
7,579993404,Kisugu,0.3097587000,32.6091322000,7401065633,0.3076673000,32.6249390000,1745.8336543244,2730.6216586954,"[579993404, 1583799967, 1583799788, 1583799935...","[32.71856677381537, 26.624433957601738, 25.730...",2730.6216586954,2735.6840000000,2840.9248157841,"LINESTRING (456506.492 34238.449, 456532.709 3...","[Kisugu, Industrial Area, Bogolobi]",3
8,1583799787,Kisugu,0.3100471000,32.6100813000,7401065633,0.3076673000,32.6249390000,1640.4063742119,2677.1014087277,"[1583799787, 6227589978, 1583800118, 158380034...","[9.721294101490457, 7.796506912348756, 12.5149...",2677.1014087277,2682.0780000000,2782.0532660583,"LINESTRING (456612.105 34270.323, 456606.586 3...","[Kisugu, Industrial Area, Bogolobi]",3
9,8144996943,Kisugu,0.3033524000,32.6059574000,8003448502,0.3007283000,32.5890897000,1892.8943434713,2774.3196406058,"[8144996943, 8144996944, 579992620, 579992609,...","[46.50197410980276, 51.71860469243541, 56.8267...",2774.3196406058,2776.6460000000,2802.9625482335,"LINESTRING (456153.187 33530.356, 456137.242 3...","[Kisugu, Kansanga-Muyenga, Kabalagala, Nsambya...",5
